In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
url = 'https://merolagani.com/CompanyDetail.aspx?symbol=GBIME#0'

In [3]:
response = requests.get(url)
response

<Response [200]>

In [4]:
response = response.content

In [5]:
soup = BeautifulSoup(response, 'html.parser')

In [6]:
soup


<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml">
<head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/><title>
	merolagani - Global IME Bank Limited (GBIME) Detail, Announcements, News, Price History, Floorsheet
</title><link href="/bundles/css/main200?v=XBmv3Nbj66DV6lzAxQCBgaIOrn903Diuut8lsSb3n9k1" rel="stylesheet"/>
<link href="/bundles/css/site205?v=JDFYx0XBsjx2PJsB70xsp6n6BXVCmnegzsdX8OWN36Y1" rel="stylesheet"/>
<link href="/Content/Services.css" rel="stylesheet"/>
<link href="/Content/owl.carousel.min.css" rel="stylesheet"/>
<link href="/Content/lightslider/css/lightslider.css" rel="stylesheet"/>
<script src="/scripts/jquery/jquery-1.11.0.min.js"></script>
<script src="/scripts/jquery/jquery-ui.js"></script>
<script src="/Content/lightslider/js/lightslider.js"></script>
<!--[if lt IE 9]>
      <script src="https://oss.maxcdn.com/html5shiv/3.7.2/html5shiv.min.js"></script>

In [7]:
table= soup.find_all('table', class_='table table-bordered table-striped table-hover')

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from selenium.common.exceptions import NoSuchElementException

symbols_list = [
    'CBBL', 'DDBL', 'FMDBL', 'KMCDB', 'NLBBL', 'NUBL', 'SKBBL', 'SLBBL', 'SMFDB', 'SWBBL',
    'MLBBL', 'LLBS', 'MMFDB', 'JSLBB', 'VLBS', 'RSDC', 'NMBMF', 'MERO', 'NADEP', 'ALBSL',
    'NMFBS', 'GMFBS', 'HLBSL', 'ILBS', 'FOWAD', 'SMATA', 'MSLB', 'GILB', 'SMB', 'GBLBS',
    'NESDO', 'MLBSL', 'MKLB', 'GLBSL', 'NICLBSL', 'SLBSL', 'SDLBSL', 'RULB', 'UNLB', 'JBLB',
    'SHLB', 'ULBSL', 'SMFBS', 'WNLB', 'SABSL', 'DLBS', 'ANLB', 'MLBS', 'AVYAN', 'JALPA',
    'ACLBSL', 'USLB', 'NSLB', 'CYCL', 'KLBSL', 'BPW', 'KLBS', 'SWMF'
]


for symbol in symbols_list:
    url = f"https://merolagani.com/CompanyDetail.aspx?symbol={symbol}"
    share_name = url.split("=")[-1]
    driver = webdriver.Chrome()  # Create a Chrome driver
    driver.get(url)  # Open the URL in the browser

    all_data = []

    price_history_tab = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'ctl00_ContentPlaceHolder1_CompanyDetail1_lnkHistoryTab'))
    )
    price_history_tab.click()
    while True:

        time.sleep(3)  # Adjust the delay as needed
        # Click on the element that loads the content (if applicable)
        # Example: Click on the "Price History" tab

        # You might need to introduce a delay here to ensure the content has loaded before fetching the HTML

        # Get the HTML content after the elements are loaded
        html_content = driver.page_source

        soup = BeautifulSoup(html_content, "html.parser")  # Parse the HTML with BeautifulSoup

        table = soup.find("table", class_="table-bordered")  # Find the table element


        each_row = table.find_all('tr')  # extract tr tag from each row

        # extract td tag text from each row excluding first row
        for row in each_row[1:]:
            td = row.find_all('td')
            row_data = [row.text for row in td]
            all_data.append(row_data)

        try:
            # Find and click the "Next" button
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[@title="Next Page"]'))
            )
            next_button.click()
        except Exception:
            break  # Break the loop if the "Next" button is not found


    driver.quit()

    df = pd.DataFrame(all_data, columns=["SN","Date", "LTP", "%change", "High", "Low", "Open","Qty.", "Turnover"])
    df.to_csv(f'company/{share_name}.csv', index=False)
    print("Finished", share_name)


Finished CBBL
Finished DDBL
Finished FMDBL
Finished KMCDB
Finished NLBBL
Finished NUBL
Finished SKBBL
Finished SLBBL
Finished SMFDB
Finished SWBBL
Finished MLBBL
Finished LLBS
Finished MMFDB
Finished JSLBB
Finished VLBS
Finished RSDC
Finished NMBMF
Finished MERO
Finished NADEP
Finished ALBSL
Finished NMFBS
Finished GMFBS
Finished HLBSL
Finished ILBS
Finished FOWAD
Finished SMATA
Finished MSLB
Finished GILB
Finished SMB
Finished GBLBS
Finished NESDO
Finished MLBSL
Finished MKLB
Finished GLBSL
Finished NICLBSL
Finished SLBSL
Finished SDLBSL
Finished RULB
Finished UNLB
Finished JBLB
Finished SHLB
Finished ULBSL
Finished SMFBS
Finished WNLB
Finished SABSL
Finished DLBS
Finished ANLB
Finished MLBS
Finished AVYAN
Finished JALPA
Finished ACLBSL
Finished USLB
Finished NSLB
Finished CYCL
Finished KLBSL
Finished BPW
Finished KLBS
Finished SWMF


In [9]:
# this is used to use this variable in the share_price.ipynb file
%store share_name


Stored 'share_name' (str)


In [10]:
share_name

'SWMF'

In [11]:
# import numpy as np
# np.shape(all_data)


In [12]:

# # Convert the collected data to a DataFrame
# df = pd.DataFrame(all_data, columns=["SN","Date", "LTP", "%change", "High", "Low", "Open","Qty.", "Turnover"])

# # Convert the DataFrame to a CSV file
# df.to_csv("stock_price.csv", index=False)